# Particle diffusion in synthetic turbulent field

In [3]:
from pylab import *

In [ ]:
from crpropa import *

## 1. Generation of turbulent and uniform fields

In [ ]:
# Grid
spacing = 10 * pc
grid_number = 256

# Turbulent field
random_seed = 42
Brms = 5 * nG
lmin = 2 * spacing
lmax = (grid_number * spacing) / 8
index = 5./3.
spectrum = SimpleTurbulenceSpectrum(Brms, lmin, lmax, index)
grid = GridProperties(Vector3d(0), grid_number, spacing)
BField = SimpleGridTurbulence(spectrum, grid, random_seed)

# Uniform field
ConstMagVec = Vector3d(0*nG,0*nG,1*nG)
BField = UniformMagneticField(ConstMagVec)

# Print field properties
print('Field properties:')
print('Lc = {:.1f} pc'.format(BField.getCorrelationLength() / pc))
print('sqrt(<B^2>) = {:.1f} nG'.format(BField.getBrms() / nG)) 
print('<|B|> = {:.1f} nG'.format(BField.getMeanFieldStrength() / nG))  

# Saving field
dumpGridToTxt(BField.getGrid(), 'kolmogorov_field.txt') 

## 2. Particle propagation

In [ ]:
N_particles = 1e5

# Propagation settings
maxlen = 100 * kpc
N = 50.
Emin = 100 * GeV
Emax = 100 * PeV
step = maxlen / N
sim = ModuleList()
sim.add(PropagationCK(BField))
sim.add(MaximumTrajectoryLength(maxlen))

# Source
source = Source()
source.add(SourcePosition(Vector3d(0.))) # Centered at origin
source.add(SourceParticleType(nucleusId(1, 1))) # Emitting protons
source.add(SourcePowerLawSpectrum(Emin, Emax, -1) ) # Energy spectrum
source.add(SourceIsotropicEmission())
print(source)

# Ouput
out = TextOutput('trajectory.txt', Output.Trajectory3D)
out.enable(Output.TrajectoryLengthColumn)
out.enable(Output.CurrentPositionColumn)
out.enable(Output.SerialNumberColumn)
out.setLengthScale(kpc)

# Observer
obs = Observer()
obs.add(ObserverTimeEvolution(step, step, N)) # Number of time steps
obs.setDeactivateOnDetection(False)
obs.onDetection(out)
sim.add(obs)

# Run simulation
sim.run(source, int(N_particles), True)
out.close()
print('Simulation finished :)')

## 3. Computation of diffusion coefficients

In [ ]:
# Load data
N_particles = int(1e3)
N_steps = int(1e4)
data = np.genfromtxt('trajectory.txt')
time = data[:,0] / 3e8 # Transform distance into time
serial_number = data[:,1] # Particle number
# Distance in all directions
x = data[:,4] 
y = data[:,5]
z = data[:,6]

# Time and space matrices
t_mat = np.zeros((N_steps, N_particles))
x_mat = np.zeros((N_steps, N_particles))
y_mat = np.zeros((N_steps, N_particles))
z_mat = np.zeros((N_steps, N_particles))
for i in range(1, N_particles+1):
    index_particles = np.where(np.logical_and(serial_number == i, serial_number == i))
    t_mat[:,i-1] = time[index_particles]
    x_mat[:,i-1] = x[index_particles]
    y_mat[:,i-1] = y[index_particles]
    z_mat[:,i-1] = z[index_particles]

In [ ]:
# Computation of dx^2, dx taken as difference from the origin
x2_mat = x_mat**2
y2_mat = y_mat**2
z2_mat = z_mat**2

# Average over all particles
dx2 = np.zeros(N_steps)
dy2 = np.zeros(N_steps)
dz2 = np.zeros(N_steps)
for j in range(0,N_particles):
    dx2 += x2_mat[:,j]
    dy2 += y2_mat[:,j]
    dz2 += z2_mat[:,j]
dx2 = dx2 / N_particles
dy2 = dy2 / N_particles
dz2 = dz2 / N_particles

# Computation of diffusion coefficients
Dxx = dx2 / (6 * t_mat[:,0])
Dyy = dy2 / (6 * t_mat[:,0])
Dzz = dz2 / (6 * t_mat[:,0])
D = Dxx + Dyy + Dzz

# Time scale in years
tau = t_mat[:,0] * 3.171e-8 

In [ ]:
# Plotting diffusion coefficients as function of time
plt.figure(figsize=(10, 7))
plt.plot(tau, Dxx, color='blue', label=r'$D_{xx}$')
plt.plot(tau, Dyy, color='red', label=r'$D_{yy}$')
plt.plot(tau, Dzz, color='green', label=r'$D_{zz}$')
plt.plot(tau, D, color='black', label=r'$D$')
plt.loglog()
plt.xlabel('Time [yrs]', fontsize=17)
plt.ylabel(r'D(E,t) [m$^2$/s]', fontsize=17)
plt.legend(ncol=1, fontsize=14)
plt.title('Diffusion coefficients', fontsize=17)
plt.savefig('diff_coeff.pdf')